In [1]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account


In [2]:
key_path = '/Users/ccvcl/Downloads/milestone-3-311019-9707de35a804.json'

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Data Extract - property sales dataset

In [3]:
data = pd.read_csv('/Volumes/LZ/Baruch College/9440/rollingsales_bronx.csv')
data1 = pd.read_csv('/Volumes/LZ/Baruch College/9440/rollingsales_brooklyn.csv')
data2 = pd.read_csv('/Volumes/LZ/Baruch College/9440/rollingsales_manhattan.csv')
data3 = pd.read_csv('/Volumes/LZ/Baruch College/9440/rollingsales_queens.csv')
data4 = pd.read_csv('/Volumes/LZ/Baruch College/9440/rollingsales_statenisland.csv')

In [4]:
dataset = pd.concat([data,data1,data2,data3,data4])

In [5]:
dataset.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY',
       'TAX CLASS AT PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING CLASS AT PRESENT', 'ADDRESS', 'APARTMENT NUMBER', 'ZIP CODE',
       'RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS',
       'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT',
       'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE',
       'SALE PRICE ', 'SALE DATE'],
      dtype='object')

In [6]:
dataset.columns = dataset.columns.str.lstrip()
dataset.columns = dataset.columns.str.rstrip()

In [7]:
dataset.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY',
       'TAX CLASS AT PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING CLASS AT PRESENT', 'ADDRESS', 'APARTMENT NUMBER', 'ZIP CODE',
       'RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS',
       'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT',
       'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE',
       'SALE PRICE', 'SALE DATE'],
      dtype='object')

In [8]:
nyc_property_sales = dataset[['BOROUGH', 'NEIGHBORHOOD','SALE PRICE','SALE DATE','ADDRESS','ZIP CODE']]

In [9]:
nyc_property_sales

,BOROUGH,NEIGHBORHOOD,SALE PRICE,SALE DATE,ADDRESS,ZIP CODE
0,2,BATHGATE,0,1/20/21,408 EAST 179TH STREET,10457.0
1,2,BATHGATE,"600,000",1/15/21,410 EAST 179TH STREET,10457.0
2,2,BATHGATE,0,7/9/20,418 EAST 180 STREET,10457.0
3,2,BATHGATE,"475,000",7/23/20,4451 PARK AVENUE,10457.0
4,2,BATHGATE,"289,000",8/25/20,2052 BATHGATE AVENUE,10457.0
...,...,...,...,...,...,...
6478,5,WOODROW,"537,000",9/25/20,98 AMBASSADOR LANE,10309.0
6479,5,WOODROW,"525,000",10/9/20,100 AMBASSADOR LANE,10309.0
6480,5,WOODROW,0,6/9/20,117 AMBASSADOR LANE,10309.0
6481,5,WOODROW,"500,000",6/2/20,72 HARRIS LANE,10309.0


# Data Cleaning


In [10]:
nyc_property_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65001 entries, 0 to 6482
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   BOROUGH       65001 non-null  int64  
 1   NEIGHBORHOOD  65001 non-null  object 
 2   SALE PRICE    65001 non-null  object 
 3   SALE DATE     65001 non-null  object 
 4   ADDRESS       65001 non-null  object 
 5   ZIP CODE      64990 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 3.5+ MB


In [11]:
#nyc_property_sales['ZIP CODE'] = nyc_property_sales['ZIP CODE'].str.rstrip()

In [15]:
nyc_property_sales = nyc_property_sales.rename(columns = {'SALE PRICE':'SALE_PRICE',
                                                          'ZIP CODE':'ZIP_CODE',
                                                         'SALE DATE': 'SALE_DATE'})

In [16]:
nyc_property_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65001 entries, 0 to 6482
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   BOROUGH       65001 non-null  int64  
 1   NEIGHBORHOOD  65001 non-null  object 
 2   SALE_PRICE    65001 non-null  object 
 3   SALE_DATE     65001 non-null  object 
 4   ADDRESS       65001 non-null  object 
 5   ZIP_CODE      64990 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 3.5+ MB


In [25]:

# change data to correct data type

nyc_property_sales['SALE_PRICE'] = nyc_property_sales['SALE_PRICE'].astype(float)


In [26]:
nyc_property_sales.isnull().sum()

BOROUGH          0
NEIGHBORHOOD     0
SALE_PRICE       0
SALE_DATE        0
ADDRESS          0
ZIP_CODE        11
dtype: int64

In [28]:
nyc_property_sales = nyc_property_sales.dropna()


In [29]:
nyc_property_sales['SALE_DATE']= pd.to_datetime(nyc_property_sales['SALE_DATE'])

nyc_property_sales['ZIP_CODE'] = nyc_property_sales['ZIP_CODE'].astype(int)

In [30]:
nyc_property_sales.shape

(64990, 6)

In [31]:
nyc_property_sales.describe()

,BOROUGH,SALE_PRICE,ZIP_CODE
count,64990.000000,6.499000e+04,64990.000000
mean,3.051885,1.148197e+06,10884.420619
std,1.250097,7.601515e+06,549.829941
min,1.000000,0.000000e+00,10001.000000
25%,2.000000,0.000000e+00,10310.000000
50%,3.000000,4.650000e+05,11211.000000
75%,4.000000,8.700000e+05,11358.000000
max,5.000000,8.099126e+08,11697.000000


In [32]:
# how many toally missing values?
#total_cells = len(nyc_property_sales.index)
#total_missing = nyc_property_sales.isnull().sum()

# percent of data missing
#(total_missing / total_cells)* 100

 ###  0 Sales Price: A 0 sale indicates that there was a transfer of ownership without a cash consideration. There can be a number of reasons for a  0 sale including transfers of ownership from parents to children. so drop it

In [33]:
#replace numbers in borough with boro name
nyc_property_sales['BOROUGH']= nyc_property_sales['BOROUGH'].astype(str)

In [34]:
nyc_property_sales['BOROUGH'] = nyc_property_sales['BOROUGH'].replace(str(1),'Manhattan')
nyc_property_sales['BOROUGH'] = nyc_property_sales['BOROUGH'].replace(str(2),'Bronx')
nyc_property_sales['BOROUGH'] = nyc_property_sales['BOROUGH'].replace(str(3),'Brooklyn')
nyc_property_sales['BOROUGH'] = nyc_property_sales['BOROUGH'].replace(str(4),'Queens')
nyc_property_sales['BOROUGH'] = nyc_property_sales['BOROUGH'].replace(str(5),'Staten Island')

In [35]:
nyc_property_sales

,BOROUGH,NEIGHBORHOOD,SALE_PRICE,SALE_DATE,ADDRESS,ZIP_CODE
0,Bronx,BATHGATE,0.0,2021-01-20,408 EAST 179TH STREET,10457
1,Bronx,BATHGATE,600000.0,2021-01-15,410 EAST 179TH STREET,10457
2,Bronx,BATHGATE,0.0,2020-07-09,418 EAST 180 STREET,10457
3,Bronx,BATHGATE,475000.0,2020-07-23,4451 PARK AVENUE,10457
4,Bronx,BATHGATE,289000.0,2020-08-25,2052 BATHGATE AVENUE,10457
...,...,...,...,...,...,...
6478,Staten Island,WOODROW,537000.0,2020-09-25,98 AMBASSADOR LANE,10309
6479,Staten Island,WOODROW,525000.0,2020-10-09,100 AMBASSADOR LANE,10309
6480,Staten Island,WOODROW,0.0,2020-06-09,117 AMBASSADOR LANE,10309
6481,Staten Island,WOODROW,500000.0,2020-06-02,72 HARRIS LANE,10309


In [36]:
#nyc_property_sales['NEIGHBORHOOD'] = nyc_property_sales['NEIGHBORHOOD'].replace(' ', '_')

In [37]:
#nyc_property_sales['NEIGHBORHOOD'].unique()

# Data extract - rent dataset

In [38]:
rent = pd.read_csv('/Volumes/LZ/Baruch College/9440/medianAskingRent_All.csv')
rent

,areaName,Borough,2020_01,2020_02,2020_03,2020_04,2020_05,2020_06,2020_07,2020_08,2020_09,2020_10,2020_11,2020_12,2021_01,2021_02,2021_03
0,All Downtown,Manhattan,4000.0,3995.0,3995.0,3850.0,3700.0,3600.0,3500.0,3380.0,3290.0,3195.0,3100.0,3100.0,3100.0,3150.0,3112.0
1,All Midtown,Manhattan,3700.0,3695.0,3685.0,3525.0,3362.0,3214.0,3098.0,2996.0,2950.0,2900.0,2877.0,2873.0,2895.0,2900.0,2950.0
2,All Upper East Side,Manhattan,3285.0,3250.0,3250.0,3150.0,3025.0,2950.0,2800.0,2700.0,2650.0,2500.0,2495.0,2495.0,2413.0,2450.0,2400.0
3,All Upper Manhattan,Manhattan,2383.0,2400.0,2450.0,2450.0,2425.0,2400.0,2400.0,2350.0,2295.0,2250.0,2208.0,2200.0,2200.0,2180.0,2195.0
4,All Upper West Side,Manhattan,3650.0,3568.0,3500.0,3400.0,3295.0,3200.0,3129.0,3000.0,2921.0,2800.0,2800.0,2800.0,2795.0,2814.0,2795.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,Windsor Terrace,Brooklyn,2500.0,2500.0,2658.0,2684.0,2545.0,2800.0,2717.0,2645.0,2645.0,2590.0,2750.0,2750.0,2750.0,2500.0,2700.0
181,Woodhaven,Queens,1800.0,1850.0,NaN,NaN,1850.0,1773.0,1838.0,2100.0,2000.0,1838.0,1923.0,2100.0,2000.0,1725.0,NaN
182,Woodlawn,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,Woodside,Queens,1950.0,2100.0,2200.0,1850.0,1833.0,1900.0,1850.0,1950.0,1850.0,1825.0,1850.0,1800.0,1775.0,1793.0,1750.0


In [39]:
rent = pd.melt(rent, id_vars = ['areaName','Borough'],value_vars = ['2020_01','2020_02','2020_03','2020_04',
                                                                  '2020_05','2020_06','2020_07','2020_08',
                                                                  '2020_09','2020_10','2020_11','2020_12',
                                                                  '2021_01','2021_02','2021_03'],
              var_name = 'date', value_name = 'rent_price')




In [40]:
rent = rent.rename(columns = {'Borough':'BOROUGH'})

In [41]:
rent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2775 entries, 0 to 2774
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   areaName    2775 non-null   object 
 1   BOROUGH     2760 non-null   object 
 2   date        2775 non-null   object 
 3   rent_price  2052 non-null   float64
dtypes: float64(1), object(3)
memory usage: 86.8+ KB


In [42]:
rent.describe()

,rent_price
count,2052.000000
mean,2505.869396
std,839.840210
min,1375.000000
25%,1944.250000
50%,2250.000000
75%,2873.250000
max,8000.000000


In [43]:
rent.isnull().sum()

areaName        0
BOROUGH        15
date            0
rent_price    723
dtype: int64

In [44]:
# how many toally missing values?
total= len(rent.index)
missing = rent.isnull().sum()

# percent of data missing
(missing / total)* 100

areaName       0.000000
BOROUGH        0.540541
date           0.000000
rent_price    26.054054
dtype: float64

In [45]:
rent = rent.dropna()
rent.isnull().sum()

areaName      0
BOROUGH       0
date          0
rent_price    0
dtype: int64

In [46]:
rent.duplicated().sum()

0

In [47]:
rent['date']=pd.to_datetime(rent['date'],format = '%Y_%m')

In [48]:
rent

,areaName,BOROUGH,date,rent_price
0,All Downtown,Manhattan,2020-01-01,4000.0
1,All Midtown,Manhattan,2020-01-01,3700.0
2,All Upper East Side,Manhattan,2020-01-01,3285.0
3,All Upper Manhattan,Manhattan,2020-01-01,2383.0
4,All Upper West Side,Manhattan,2020-01-01,3650.0
...,...,...,...,...
2768,Williamsbridge,Bronx,2021-03-01,1950.0
2769,Williamsburg,Brooklyn,2021-03-01,2800.0
2770,Windsor Terrace,Brooklyn,2021-03-01,2700.0
2773,Woodside,Queens,2021-03-01,1750.0


# Data extract - COVID dataset

In [49]:

df2 = pd.read_csv('/Volumes/LZ/Baruch College/9440/covid-cases-by-day.csv')
df3 = pd.read_csv('/Volumes/LZ/Baruch College/9440/covid-deaths-by-day.csv')
df4 = pd.read_csv('/Volumes/LZ/Baruch College/9440/data-by-modzcta.csv')

In [50]:
df4

,MODIFIED_ZCTA,NEIGHBORHOOD_NAME,BOROUGH_GROUP,label,lat,lon,COVID_CASE_COUNT,COVID_CASE_RATE,POP_DENOMINATOR,COVID_DEATH_COUNT,COVID_DEATH_RATE,PERCENT_POSITIVE,TOTAL_COVID_TESTS
0,10001,Chelsea/NoMad/West Chelsea,Manhattan,"10001, 10118",40.750693,-73.997137,1586,5743.65,27613.09,35,126.75,7.82,20950
1,10002,Chinatown/Lower East Side,Manhattan,10002,40.715781,-73.986176,6046,8026.80,75322.71,274,363.77,12.60,49686
2,10003,East Village/Gramercy/Greenwich Village,Manhattan,10003,40.731825,-73.989164,2893,5359.61,53977.81,48,88.93,7.05,41899
3,10004,Financial District,Manhattan,10004,40.703675,-74.013106,250,8411.52,2972.12,2,67.29,6.84,3685
4,10005,Financial District,Manhattan,10005,40.706092,-74.008861,423,4830.30,8757.23,0,0.00,6.67,6326
...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,11691,Edgemere/Far Rockaway,Queens,11691,40.601330,-73.761357,7424,11104.41,66856.31,455,680.56,17.54,44722
173,11692,Arverne/Edgemere,Queens,11692,40.592794,-73.792203,1844,8768.79,21029.13,105,499.31,15.17,12969
174,11693,Arverne/Broad Channel,Queens,11693,40.603699,-73.821926,1135,8956.66,12672.14,39,307.76,15.90,7333
175,11694,Belle Harbor-Neponsit/Rockaway Park,Queens,11694,40.577599,-73.844553,2261,10880.97,20779.40,96,462.00,16.65,14058


In [51]:
covid_cases_day = df2

In [52]:
covid_cases_day.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431 entries, 0 to 430
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   date           431 non-null    object
 1   Bronx          431 non-null    int64 
 2   Brooklyn       431 non-null    int64 
 3   Manhattan      431 non-null    int64 
 4   Queens         431 non-null    int64 
 5   Staten Island  431 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 20.3+ KB


In [53]:
covid_cases_day.isnull().sum()

date             0
Bronx            0
Brooklyn         0
Manhattan        0
Queens           0
Staten Island    0
dtype: int64

In [54]:
covid_cases_day['date']= pd.to_datetime(covid_cases_day['date'])

In [55]:
covid_cases_day.columns


Index(['date', 'Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island'], dtype='object')

In [56]:
covid_cases_day = pd.melt(covid_cases_day, id_vars = ['date'],value_vars = ['Bronx','Brooklyn','Manhattan','Queens','Staten Island'],
                         var_name = 'BOROUGH',value_name ='cases' )

In [57]:
covid_cases_day

,date,BOROUGH,cases
0,2020-02-29,Bronx,0
1,2020-03-01,Bronx,0
2,2020-03-02,Bronx,0
3,2020-03-03,Bronx,0
4,2020-03-04,Bronx,0
...,...,...,...
2150,2021-04-30,Staten Island,73
2151,2021-05-01,Staten Island,56
2152,2021-05-02,Staten Island,61
2153,2021-05-03,Staten Island,62


In [58]:
covid_death_day = df3

In [59]:
covid_death_day.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431 entries, 0 to 430
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   date           431 non-null    object
 1   Bronx          431 non-null    int64 
 2   Brooklyn       431 non-null    int64 
 3   Manhattan      431 non-null    int64 
 4   Queens         431 non-null    int64 
 5   Staten_Island  431 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 20.3+ KB


In [60]:
covid_death_day['date']= pd.to_datetime(covid_cases_day['date'])

In [61]:
covid_death_day=covid_death_day.rename(columns={'Staten_Island':'Staten Island'})

In [62]:
covid_death_day.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431 entries, 0 to 430
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           431 non-null    datetime64[ns]
 1   Bronx          431 non-null    int64         
 2   Brooklyn       431 non-null    int64         
 3   Manhattan      431 non-null    int64         
 4   Queens         431 non-null    int64         
 5   Staten Island  431 non-null    int64         
dtypes: datetime64[ns](1), int64(5)
memory usage: 20.3 KB


In [63]:
covid_death_day = pd.melt(covid_death_day, id_vars = ['date'],value_vars = ['Bronx','Brooklyn','Manhattan','Queens','Staten Island'],
                         var_name = 'BOROUGH', value_name = 'death_per_day')

In [64]:
covid_death_day

,date,BOROUGH,death_per_day
0,2020-02-29,Bronx,0
1,2020-03-01,Bronx,0
2,2020-03-02,Bronx,0
3,2020-03-03,Bronx,0
4,2020-03-04,Bronx,0
...,...,...,...
2150,2021-04-30,Staten Island,3
2151,2021-05-01,Staten Island,2
2152,2021-05-02,Staten Island,2
2153,2021-05-03,Staten Island,4


In [65]:
covid_death_day.isnull().sum()

date             0
BOROUGH          0
death_per_day    0
dtype: int64

In [66]:
covid_cases_day

,date,BOROUGH,cases
0,2020-02-29,Bronx,0
1,2020-03-01,Bronx,0
2,2020-03-02,Bronx,0
3,2020-03-03,Bronx,0
4,2020-03-04,Bronx,0
...,...,...,...
2150,2021-04-30,Staten Island,73
2151,2021-05-01,Staten Island,56
2152,2021-05-02,Staten Island,61
2153,2021-05-03,Staten Island,62


In [67]:
covid_cases_day = covid_cases_day.merge(covid_death_day, left_on=['date','BOROUGH'],right_on = ['date','BOROUGH'], how = 'inner')

In [68]:
covid_cases_day = covid_cases_day.rename(columns={'cases':'cases_per_day'})

In [69]:
covid_cases_day

,date,BOROUGH,cases_per_day,death_per_day
0,2020-02-29,Bronx,0,0
1,2020-03-01,Bronx,0,0
2,2020-03-02,Bronx,0,0
3,2020-03-03,Bronx,0,0
4,2020-03-04,Bronx,0,0
...,...,...,...,...
2150,2021-04-30,Staten Island,73,3
2151,2021-05-01,Staten Island,56,2
2152,2021-05-02,Staten Island,61,2
2153,2021-05-03,Staten Island,62,4


In [70]:
covid_zip = df4

In [71]:
covid_zip.isnull().sum()

MODIFIED_ZCTA        0
NEIGHBORHOOD_NAME    0
BOROUGH_GROUP        0
label                0
lat                  0
lon                  0
COVID_CASE_COUNT     0
COVID_CASE_RATE      0
POP_DENOMINATOR      0
COVID_DEATH_COUNT    0
COVID_DEATH_RATE     0
PERCENT_POSITIVE     0
TOTAL_COVID_TESTS    0
dtype: int64

In [72]:
covid_zip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   MODIFIED_ZCTA      177 non-null    int64  
 1   NEIGHBORHOOD_NAME  177 non-null    object 
 2   BOROUGH_GROUP      177 non-null    object 
 3   label              177 non-null    object 
 4   lat                177 non-null    float64
 5   lon                177 non-null    float64
 6   COVID_CASE_COUNT   177 non-null    int64  
 7   COVID_CASE_RATE    177 non-null    float64
 8   POP_DENOMINATOR    177 non-null    float64
 9   COVID_DEATH_COUNT  177 non-null    int64  
 10  COVID_DEATH_RATE   177 non-null    float64
 11  PERCENT_POSITIVE   177 non-null    float64
 12  TOTAL_COVID_TESTS  177 non-null    int64  
dtypes: float64(6), int64(4), object(3)
memory usage: 18.1+ KB


# Create property dimension

In [75]:
location_dim = nyc_property_sales.copy()
location_dim = location_dim[['BOROUGH','NEIGHBORHOOD','ADDRESS','ZIP_CODE']]

In [76]:
location_dim

,BOROUGH,NEIGHBORHOOD,ADDRESS,ZIP_CODE
0,Bronx,BATHGATE,408 EAST 179TH STREET,10457
1,Bronx,BATHGATE,410 EAST 179TH STREET,10457
2,Bronx,BATHGATE,418 EAST 180 STREET,10457
3,Bronx,BATHGATE,4451 PARK AVENUE,10457
4,Bronx,BATHGATE,2052 BATHGATE AVENUE,10457
...,...,...,...,...
6478,Staten Island,WOODROW,98 AMBASSADOR LANE,10309
6479,Staten Island,WOODROW,100 AMBASSADOR LANE,10309
6480,Staten Island,WOODROW,117 AMBASSADOR LANE,10309
6481,Staten Island,WOODROW,72 HARRIS LANE,10309


In [77]:
location_dim = location_dim.drop_duplicates(subset = ['ZIP_CODE'],keep = 'first')
location_dim

,BOROUGH,NEIGHBORHOOD,ADDRESS,ZIP_CODE
0,Bronx,BATHGATE,408 EAST 179TH STREET,10457
7,Bronx,BATHGATE,1465 CROTONA PLACE,10456
14,Bronx,BATHGATE,444 EAST 186 STREET,10458
64,Bronx,BAYCHESTER,1114 EAST 212 STREET,10469
138,Bronx,BAYCHESTER,1314 EAST 223 STREET,10466
...,...,...,...,...
914,Staten Island,CLOVE LAKES,76 BEVERLY AVENUE,10301
1002,Staten Island,CONCORD,315 GORDON STREET,10304
1582,Staten Island,GRANT CITY,1847 RICHMOND ROAD,10306
1817,Staten Island,GREAT KILLS,11 TANGLEWOOD DRIVE,10308


In [78]:
location_dim['boro_id']= ''

<ipython-input-78-999626b61545>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_dim['boro_id']= ''


In [79]:
location_dim['boro_id'][location_dim.BOROUGH == 'Bronx'] = 1
location_dim['boro_id'][location_dim.BOROUGH == 'Brooklyn'] = 2
location_dim['boro_id'][location_dim.BOROUGH == 'Manhattan'] = 3
location_dim['boro_id'][location_dim.BOROUGH == 'Queens'] = 4
location_dim['boro_id'][location_dim.BOROUGH == 'Staten Island'] = 5


<ipython-input-79-d49e280bb247>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_dim['boro_id'][location_dim.BOROUGH == 'Bronx'] = 1
/Users/ccvcl/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:1020: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)
<ipython-input-79-d49e280bb247>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_dim['boro_id'][location_dim.BOROUGH == 

In [80]:
location_dim.insert(0,'zip_id',range(1000, 1000+len(location_dim)))
location_dim

,zip_id,BOROUGH,NEIGHBORHOOD,ADDRESS,ZIP_CODE,boro_id
0,1000,Bronx,BATHGATE,408 EAST 179TH STREET,10457,1
7,1001,Bronx,BATHGATE,1465 CROTONA PLACE,10456,1
14,1002,Bronx,BATHGATE,444 EAST 186 STREET,10458,1
64,1003,Bronx,BAYCHESTER,1114 EAST 212 STREET,10469,1
138,1004,Bronx,BAYCHESTER,1314 EAST 223 STREET,10466,1
...,...,...,...,...,...,...
914,1177,Staten Island,CLOVE LAKES,76 BEVERLY AVENUE,10301,5
1002,1178,Staten Island,CONCORD,315 GORDON STREET,10304,5
1582,1179,Staten Island,GRANT CITY,1847 RICHMOND ROAD,10306,5
1817,1180,Staten Island,GREAT KILLS,11 TANGLEWOOD DRIVE,10308,5


In [81]:
# merge the dimension to raw data, left join keep everything on left and join the value on right table

nyc_property_sales = nyc_property_sales.merge(location_dim[['zip_id','ZIP_CODE']],left_on = 'ZIP_CODE',
                                             right_on = 'ZIP_CODE', how = 'left')

In [82]:
nyc_property_sales = nyc_property_sales.merge(location_dim[['boro_id','BOROUGH']],left_on = 'BOROUGH',
                                             right_on = 'BOROUGH', how = 'left')

In [83]:
nyc_property_sales

,BOROUGH,NEIGHBORHOOD,SALE_PRICE,SALE_DATE,ADDRESS,ZIP_CODE,zip_id,boro_id
0,Bronx,BATHGATE,0.0,2021-01-20,408 EAST 179TH STREET,10457,1000,1
1,Bronx,BATHGATE,0.0,2021-01-20,408 EAST 179TH STREET,10457,1000,1
2,Bronx,BATHGATE,0.0,2021-01-20,408 EAST 179TH STREET,10457,1000,1
3,Bronx,BATHGATE,0.0,2021-01-20,408 EAST 179TH STREET,10457,1000,1
4,Bronx,BATHGATE,0.0,2021-01-20,408 EAST 179TH STREET,10457,1000,1
...,...,...,...,...,...,...,...,...
2825328,Staten Island,WOODROW,500000.0,2020-06-02,74 HARRIS LANE,10309,1171,5
2825329,Staten Island,WOODROW,500000.0,2020-06-02,74 HARRIS LANE,10309,1171,5
2825330,Staten Island,WOODROW,500000.0,2020-06-02,74 HARRIS LANE,10309,1171,5
2825331,Staten Island,WOODROW,500000.0,2020-06-02,74 HARRIS LANE,10309,1171,5


In [84]:
covid_cases_day = covid_cases_day.merge(location_dim[['boro_id','BOROUGH']],left_on = 'BOROUGH',
                                        right_on = 'BOROUGH', how = 'left')

In [85]:
covid_cases_day

,date,BOROUGH,cases_per_day,death_per_day,boro_id
0,2020-02-29,Bronx,0,0,1
1,2020-02-29,Bronx,0,0,1
2,2020-02-29,Bronx,0,0,1
3,2020-02-29,Bronx,0,0,1
4,2020-02-29,Bronx,0,0,1
...,...,...,...,...,...
78437,2021-05-04,Staten Island,60,4,5
78438,2021-05-04,Staten Island,60,4,5
78439,2021-05-04,Staten Island,60,4,5
78440,2021-05-04,Staten Island,60,4,5


In [86]:
rent = rent.merge(location_dim[['boro_id','BOROUGH']],left_on = 'BOROUGH',
                                        right_on = 'BOROUGH', how = 'left')

In [87]:
#covid_cases_day['boro_id']=covid_cases_day['Borough'].apply(lambda x: property_dim[[covid_cases_day['Borough']== x]]['boro_id'])

In [88]:
rent

,areaName,BOROUGH,date,rent_price,boro_id
0,All Downtown,Manhattan,2020-01-01,4000.0,3
1,All Downtown,Manhattan,2020-01-01,4000.0,3
2,All Downtown,Manhattan,2020-01-01,4000.0,3
3,All Downtown,Manhattan,2020-01-01,4000.0,3
4,All Downtown,Manhattan,2020-01-01,4000.0,3
...,...,...,...,...,...
88403,Woodstock,Bronx,2021-03-01,1375.0,1
88404,Woodstock,Bronx,2021-03-01,1375.0,1
88405,Woodstock,Bronx,2021-03-01,1375.0,1
88406,Woodstock,Bronx,2021-03-01,1375.0,1


In [89]:
covid_zip = covid_zip.merge(location_dim[['zip_id','ZIP_CODE']],left_on = 'MODIFIED_ZCTA',
                                             right_on = 'ZIP_CODE', how = 'left')

In [116]:
covid_zip = covid_zip.merge(location_dim[['boro_id','BOROUGH']],left_on = 'BOROUGH_GROUP',
                                        right_on = 'BOROUGH', how = 'left')

In [117]:
covid_zip.isnull().sum()

MODIFIED_ZCTA        0
NEIGHBORHOOD_NAME    0
BOROUGH_GROUP        0
label                0
lat                  0
lon                  0
COVID_CASE_COUNT     0
COVID_CASE_RATE      0
POP_DENOMINATOR      0
COVID_DEATH_COUNT    0
COVID_DEATH_RATE     0
PERCENT_POSITIVE     0
TOTAL_COVID_TESTS    0
zip_id               0
ZIP_CODE             0
boro_id              0
BOROUGH              0
dtype: int64

In [118]:
covid_zip.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7821 entries, 0 to 7820
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   MODIFIED_ZCTA      7821 non-null   int64  
 1   NEIGHBORHOOD_NAME  7821 non-null   object 
 2   BOROUGH_GROUP      7821 non-null   object 
 3   label              7821 non-null   object 
 4   lat                7821 non-null   float64
 5   lon                7821 non-null   float64
 6   COVID_CASE_COUNT   7821 non-null   int64  
 7   COVID_CASE_RATE    7821 non-null   float64
 8   POP_DENOMINATOR    7821 non-null   float64
 9   COVID_DEATH_COUNT  7821 non-null   int64  
 10  COVID_DEATH_RATE   7821 non-null   float64
 11  PERCENT_POSITIVE   7821 non-null   float64
 12  TOTAL_COVID_TESTS  7821 non-null   int64  
 13  zip_id             7821 non-null   int64  
 14  ZIP_CODE           7821 non-null   int64  
 15  boro_id            7821 non-null   object 
 16  BOROUGH            7821 

In [119]:
#covid_zip['zip_id']= covid_zip['zip_id'].astype(int)
#covid_zip['ZIP_CODE']= covid_zip['ZIP_CODE'].astype(int)

In [120]:
covid_zip

,MODIFIED_ZCTA,NEIGHBORHOOD_NAME,BOROUGH_GROUP,label,lat,lon,COVID_CASE_COUNT,COVID_CASE_RATE,POP_DENOMINATOR,COVID_DEATH_COUNT,COVID_DEATH_RATE,PERCENT_POSITIVE,TOTAL_COVID_TESTS,zip_id,ZIP_CODE,boro_id,BOROUGH
0,10001,Chelsea/NoMad/West Chelsea,Manhattan,"10001, 10118",40.750693,-73.997137,1586,5743.65,27613.09,35,126.75,7.82,20950,1067,10001,3,Manhattan
1,10001,Chelsea/NoMad/West Chelsea,Manhattan,"10001, 10118",40.750693,-73.997137,1586,5743.65,27613.09,35,126.75,7.82,20950,1067,10001,3,Manhattan
2,10001,Chelsea/NoMad/West Chelsea,Manhattan,"10001, 10118",40.750693,-73.997137,1586,5743.65,27613.09,35,126.75,7.82,20950,1067,10001,3,Manhattan
3,10001,Chelsea/NoMad/West Chelsea,Manhattan,"10001, 10118",40.750693,-73.997137,1586,5743.65,27613.09,35,126.75,7.82,20950,1067,10001,3,Manhattan
4,10001,Chelsea/NoMad/West Chelsea,Manhattan,"10001, 10118",40.750693,-73.997137,1586,5743.65,27613.09,35,126.75,7.82,20950,1067,10001,3,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7816,11697,Breezy Point,Queens,11697,40.559070,-73.907199,559,16476.96,3392.62,12,353.71,15.64,3638,1128,11697,4,Queens
7817,11697,Breezy Point,Queens,11697,40.559070,-73.907199,559,16476.96,3392.62,12,353.71,15.64,3638,1128,11697,4,Queens
7818,11697,Breezy Point,Queens,11697,40.559070,-73.907199,559,16476.96,3392.62,12,353.71,15.64,3638,1128,11697,4,Queens
7819,11697,Breezy Point,Queens,11697,40.559070,-73.907199,559,16476.96,3392.62,12,353.71,15.64,3638,1128,11697,4,Queens


# Create date dimension

In [95]:
date1 = nyc_property_sales.copy()
date2 = covid_cases_day.copy()
date3 = rent.copy()

In [97]:
date1 = nyc_property_sales[['SALE_DATE']]

# rename the column name fo nyc property date, then concat
date1 = date1.rename(columns = {'SALE_DATE':'date'})
date2 = covid_cases_day[['date']]
date3 = rent[['date']]

In [98]:
date_dim = pd.concat([date1,date2,date3])

In [99]:
date_dim

,date
0,2021-01-20
1,2021-01-20
2,2021-01-20
3,2021-01-20
4,2021-01-20
...,...
88403,2021-03-01
88404,2021-03-01
88405,2021-03-01
88406,2021-03-01


In [100]:
date_dim = date_dim.drop_duplicates(subset= 'date', keep = 'first')

In [101]:
date_dim

,date
0,2021-01-20
25,2021-01-15
50,2020-07-09
75,2020-07-23
100,2020-08-25
...,...
10700,2021-05-02
10725,2021-05-03
10750,2021-05-04
0,2020-01-01


In [102]:
date_dim.insert(0, 'date_id',range(100, 100+len(date_dim)))

In [103]:
date_dim

,date_id,date
0,100,2021-01-20
25,101,2021-01-15
50,102,2020-07-09
75,103,2020-07-23
100,104,2020-08-25
...,...,...
10700,528,2021-05-02
10725,529,2021-05-03
10750,530,2021-05-04
0,531,2020-01-01


In [104]:
# get year, month, day from datetime

date_dim['Year']= date_dim['date'].dt.year
date_dim['Month']= date_dim['date'].dt.month
date_dim['Day']= date_dim['date'].dt.day
date_dim

<ipython-input-104-7f363f4024d2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_dim['Year']= date_dim['date'].dt.year
<ipython-input-104-7f363f4024d2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_dim['Month']= date_dim['date'].dt.month
<ipython-input-104-7f363f4024d2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

,date_id,date,Year,Month,Day
0,100,2021-01-20,2021,1,20
25,101,2021-01-15,2021,1,15
50,102,2020-07-09,2020,7,9
75,103,2020-07-23,2020,7,23
100,104,2020-08-25,2020,8,25
...,...,...,...,...,...
10700,528,2021-05-02,2021,5,2
10725,529,2021-05-03,2021,5,3
10750,530,2021-05-04,2021,5,4
0,531,2020-01-01,2020,1,1


In [105]:
# merge date_dim to raw data
nyc_property_sales = nyc_property_sales.merge(date_dim[['date_id','date']],left_on = 'SALE_DATE',
                                             right_on = 'date', how = 'left')
nyc_property_sales

,BOROUGH,NEIGHBORHOOD,SALE_PRICE,SALE_DATE,ADDRESS,ZIP_CODE,zip_id,boro_id,date_id,date
0,Bronx,BATHGATE,0.0,2021-01-20,408 EAST 179TH STREET,10457,1000,1,100,2021-01-20
1,Bronx,BATHGATE,0.0,2021-01-20,408 EAST 179TH STREET,10457,1000,1,100,2021-01-20
2,Bronx,BATHGATE,0.0,2021-01-20,408 EAST 179TH STREET,10457,1000,1,100,2021-01-20
3,Bronx,BATHGATE,0.0,2021-01-20,408 EAST 179TH STREET,10457,1000,1,100,2021-01-20
4,Bronx,BATHGATE,0.0,2021-01-20,408 EAST 179TH STREET,10457,1000,1,100,2021-01-20
...,...,...,...,...,...,...,...,...,...,...
2825328,Staten Island,WOODROW,500000.0,2020-06-02,74 HARRIS LANE,10309,1171,5,272,2020-06-02
2825329,Staten Island,WOODROW,500000.0,2020-06-02,74 HARRIS LANE,10309,1171,5,272,2020-06-02
2825330,Staten Island,WOODROW,500000.0,2020-06-02,74 HARRIS LANE,10309,1171,5,272,2020-06-02
2825331,Staten Island,WOODROW,500000.0,2020-06-02,74 HARRIS LANE,10309,1171,5,272,2020-06-02


In [106]:
covid_cases_day = covid_cases_day.merge(date_dim[['date_id','date']],left_on = 'date',
                                             right_on = 'date', how = 'left')
rent = rent.merge(date_dim[['date_id','date']],left_on = 'date',
                                             right_on = 'date', how = 'left')

In [107]:
covid_cases_day 

,date,BOROUGH,cases_per_day,death_per_day,boro_id,date_id
0,2020-02-29,Bronx,0,0,1,465
1,2020-02-29,Bronx,0,0,1,465
2,2020-02-29,Bronx,0,0,1,465
3,2020-02-29,Bronx,0,0,1,465
4,2020-02-29,Bronx,0,0,1,465
...,...,...,...,...,...,...
78437,2021-05-04,Staten Island,60,4,5,530
78438,2021-05-04,Staten Island,60,4,5,530
78439,2021-05-04,Staten Island,60,4,5,530
78440,2021-05-04,Staten Island,60,4,5,530


In [108]:
rent

,areaName,BOROUGH,date,rent_price,boro_id,date_id
0,All Downtown,Manhattan,2020-01-01,4000.0,3,531
1,All Downtown,Manhattan,2020-01-01,4000.0,3,531
2,All Downtown,Manhattan,2020-01-01,4000.0,3,531
3,All Downtown,Manhattan,2020-01-01,4000.0,3,531
4,All Downtown,Manhattan,2020-01-01,4000.0,3,531
...,...,...,...,...,...,...
88403,Woodstock,Bronx,2021-03-01,1375.0,1,153
88404,Woodstock,Bronx,2021-03-01,1375.0,1,153
88405,Woodstock,Bronx,2021-03-01,1375.0,1,153
88406,Woodstock,Bronx,2021-03-01,1375.0,1,153


# Create Sales Fact Table

In [109]:
# sales_fact
sales_fact = nyc_property_sales.copy()
sales_fact = sales_fact[['boro_id','zip_id','date_id','SALE_PRICE']]
sales_fact

,boro_id,zip_id,date_id,SALE_PRICE
0,1,1000,100,0.0
1,1,1000,100,0.0
2,1,1000,100,0.0
3,1,1000,100,0.0
4,1,1000,100,0.0
...,...,...,...,...
2825328,5,1171,272,500000.0
2825329,5,1171,272,500000.0
2825330,5,1171,272,500000.0
2825331,5,1171,272,500000.0


# Creat Rent Fact Table

In [110]:
rent_fact = rent.copy()
rent_fact = rent_fact[['rent_price','boro_id','date_id']]

In [111]:
rent_fact

,rent_price,boro_id,date_id
0,4000.0,3,531
1,4000.0,3,531
2,4000.0,3,531
3,4000.0,3,531
4,4000.0,3,531
...,...,...,...
88403,1375.0,1,153
88404,1375.0,1,153
88405,1375.0,1,153
88406,1375.0,1,153


# Creat Covid Fact Table

In [125]:
covid1 = covid_cases_day.copy()
covid_day_fact = covid1[['boro_id','date_id','cases_per_day','death_per_day']]

In [126]:
covid_day_fact

,boro_id,date_id,cases_per_day,death_per_day
0,1,465,0,0
1,1,465,0,0
2,1,465,0,0
3,1,465,0,0
4,1,465,0,0
...,...,...,...,...
78437,5,530,60,4
78438,5,530,60,4
78439,5,530,60,4
78440,5,530,60,4


In [127]:
covid_zip_fact = covid_zip.copy()
covid_zip_fact = covid_zip_fact[['zip_id','ZIP_CODE','COVID_CASE_COUNT','COVID_CASE_RATE','POP_DENOMINATOR',
                                'COVID_DEATH_COUNT','COVID_DEATH_RATE','PERCENT_POSITIVE','TOTAL_COVID_TESTS']]

In [128]:
covid_zip_fact

,zip_id,ZIP_CODE,COVID_CASE_COUNT,COVID_CASE_RATE,POP_DENOMINATOR,COVID_DEATH_COUNT,COVID_DEATH_RATE,PERCENT_POSITIVE,TOTAL_COVID_TESTS
0,1067,10001,1586,5743.65,27613.09,35,126.75,7.82,20950
1,1067,10001,1586,5743.65,27613.09,35,126.75,7.82,20950
2,1067,10001,1586,5743.65,27613.09,35,126.75,7.82,20950
3,1067,10001,1586,5743.65,27613.09,35,126.75,7.82,20950
4,1067,10001,1586,5743.65,27613.09,35,126.75,7.82,20950
...,...,...,...,...,...,...,...,...,...
7816,1128,11697,559,16476.96,3392.62,12,353.71,15.64,3638
7817,1128,11697,559,16476.96,3392.62,12,353.71,15.64,3638
7818,1128,11697,559,16476.96,3392.62,12,353.71,15.64,3638
7819,1128,11697,559,16476.96,3392.62,12,353.71,15.64,3638


# Load data

In [129]:

dataset_id = 'milestone-3-311019.nyc_property_sales'

dataset_ref = client.dataset(dataset_id)

job_config = bigquery.LoadJobConfig()
job_config.autodetect = True
job_config.write_disposition = "WRITE_TRUNCATE"
    


load_job = client.load_table_from_dataframe(location_dim,'nyc_property_sales.location_dim',job_config=job_config)
load_job = client.load_table_from_dataframe(date_dim,'nyc_property_sales.date_dim',job_config=job_config)
load_job = client.load_table_from_dataframe(sales_fact,'nyc_property_sales.sales_fact',job_config=job_config)
load_job = client.load_table_from_dataframe(rent_fact,'nyc_property_sales.rent_fact',job_config=job_config)
load_job = client.load_table_from_dataframe(covid_day_fact,'nyc_property_sales.covid_day_fact',job_config=job_config)
load_job = client.load_table_from_dataframe(covid_zip_fact,'nyc_property_sales.covid_zip_fact',job_config=job_config)


print("Starting job {}".format(load_job))
print('Done')

Starting job <google.cloud.bigquery.job.load.LoadJob object at 0x7ffcfbc96610>
Done
